- This code refers to Kaggle kernal: https://www.kaggle.com/frankherfert/tips-tricks-for-working-with-large-datasets?source=post_page---------------------------

In [14]:
import pandas as pd
import os
#We will use titanic dataset for demonstration

In [5]:
%%time
df = pd.read_csv('train.csv')
#Time magic function let us know the time this code spend

Wall time: 4.88 ms


In [6]:
#Checking out the memory usage, this one takes 318.5 kb
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 318.5 KB


In [7]:
#Memory usage in MB
df.memory_usage(deep=True) * 1e-6

Index          0.000080
PassengerId    0.007128
Survived       0.007128
Pclass         0.007128
Name           0.074813
Sex            0.054979
Age            0.007128
SibSp          0.007128
Parch          0.007128
Ticket         0.056802
Fare           0.007128
Cabin          0.034360
Embarked       0.055182
dtype: float64

In [9]:
#Total memory usage:
df.memory_usage(deep=True).sum() * 1e-6

0.326112

In [10]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Tip 1. convert date time column to using pd to date
- Saves lots of memory

In [ ]:
print("size before:", train["activation_date"].memory_usage(deep=True) * 1e-6)
df["date column"] = pd.to_datetime(df["date column"])
print("size after: ", train["activation_date"].memory_usage(deep=True) * 1e-6)

## Tip 2. convert categorical column to categorical
- Saves lots of memory

In [12]:
print("size before:", df["Sex"].memory_usage(deep=True))
df["Sex"] = df["Sex"].astype("category")
print("size after :", df["Sex"].memory_usage(deep=True))

size before: 55059
size after : 1175


In [13]:
#Resuable function
def convert_columns_to_catg(df, column_list):
    '''
    Read in a pandas dataframe called df, and a list of columns that you would like to convert to categorical
    returns the memory before and after converting in MBs
    '''
    for col in column_list:
        print("converting", col.ljust(30), "size: ", round(df[col].memory_usage(deep=True)*1e-6,2), end="\t")
        df[col] = df[col].astype("category")
        print("->\t", round(df[col].memory_usage(deep=True)*1e-6,2))

## Tip 3. Saving files to pickle file to faster load up

- This dataset is not a great demonstration of the procedure since it is relatively small

In [15]:
df.to_pickle("train.pkl")

print("train.csv:", os.stat('train.csv').st_size * 1e-6)
print("train.pkl:", os.stat('train.pkl').st_size * 1e-6)

train.csv: 0.061194
train.pkl: 0.098674


In [16]:
%%time
new_df = pd.read_pickle("train.pkl")

Wall time: 4.88 ms


In [17]:
os.remove("train.pkl")

## Tip 4. Garbage collector

- Python has a library for controlling it's garbace collector, a system to manage objects in memory and specifically removing unneeded objects.

- After doing larger transformations, object creations/deletions or generally anything else that runs for more than a few seconds,, it can help to free up memory by calling the garbage collector directly.

- On your own computer you can use htop, the Windows taks manager and similar tools to monitor the RAM, for demonstration in this notebook we can use psutil to show the used RAM.

- Homepage: https://docs.python.org/3/library/gc.html
- gc.enable(): Enable automatic garbage collection.

- gc.disable(): Disable automatic garbage collection.

- gc.isenabled(): Returns true if automatic collection is enabled.

- gc.collect(generation=2: With no arguments, run a full collection. The optional argument generation may be an integer specifying which generation to collect (from 0 to 2). A ValueError is raised if the generation number is invalid. The number of unreachable objects found is returned.


In [18]:
import gc
import psutil

In [19]:
print("available RAM:", psutil.virtual_memory())

gc.collect()

print("available RAM:", psutil.virtual_memory())

available RAM: svmem(total=34320764928, available=22615969792, percent=34.1, used=11704795136, free=22615969792)
available RAM: svmem(total=34320764928, available=22616526848, percent=34.1, used=11704238080, free=22616526848)


## Tip 5. Downcast Integer columns

- We can see that pandas created all new columns as int64, meaning they can store values between -9223372036854775808 to 9223372036854775807. We do not need that

In [20]:
def downcast_df_int_columns(df):
    list_of_columns = list(df.select_dtypes(include=["int32", "int64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting integers for:", list_of_columns, "\n")
        
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="integer")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
    
    gc.collect()
    
    print("done")

In [21]:
downcast_df_int_columns(df)

downcasting integers for: ['PassengerId', 'Survived', 'Pclass', 'SibSp', 'Parch'] 

reduced memory usage for:   PassengerId   from     0.01 to      0.0
reduced memory usage for:   Survived      from     0.01 to      0.0
reduced memory usage for:   Pclass        from     0.01 to      0.0
reduced memory usage for:   SibSp         from     0.01 to      0.0
reduced memory usage for:   Parch         from     0.01 to      0.0
done


In [22]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int16
Survived       891 non-null int8
Pclass         891 non-null int8
Name           891 non-null object
Sex            891 non-null category
Age            714 non-null float64
SibSp          891 non-null int8
Parch          891 non-null int8
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: category(1), float64(2), int16(1), int8(4), object(4)
memory usage: 48.0+ KB
